In [1]:
## If you run for first time uncomment these
# !pip install elasticsearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from scipy import spatial
from elasticsearch import Elasticsearch

In [2]:
#Parameters
no_of_recepies = 10000 #124473
ingredient_list = 'Filterd_ingredient' # ingredients (otherwise)

Reading Pickel File for Filtered Data and creating dictionary of recipes

In [3]:
%%time
# Gagan's Code
import pandas as pd
rec_df = pd.read_pickle("Cleaned_recipes.P")
rec_l = []
temp_dict = rec_df.to_dict(orient='index')
for ind in temp_dict:
    rec_l.append(temp_dict[ind])
es = Elasticsearch()
for i in range(0,len(rec_l[:no_of_recepies])):
    res = es.index(index='recipes', doc_type='recipe', id=i, body=rec_l[i])

CPU times: user 15.7 s, sys: 1.79 s, total: 17.5 s
Wall time: 3min 19s


In [9]:
max_res = 10000

In [32]:
rec_df.head()
# %%time
# recipes = []
# with open('recipes_raw_nosource_ar.json') as json_file: #make sure the recipe json is in same dir
#     data = json.load(json_file)
#     for p in data:
#         recipes.append(data[p])
# es = Elasticsearch()
# for i in range(0,len(recipes[:500])): #i took only first 120 recipes because time and size
#     res = es.index(index='recipes', doc_type='recipe', id=i, body=recipes[i])

,ingredients,instructions,picture_link,title,filtered_ingredient,Filterd_ingredient
rmK12Uau.ntP510KeImX506H6Mr6jTu,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S,Slow Cooker Chicken and Dumplings,"[skinless, boneless chicken breast, butter, co...","[skinless boneless chicken breast, butter, con..."
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,Awesome Slow Cooker Pot Roast,"[condensed cream of mushroom soup, dry onion s...","[condensed cream mushroom soup, dry onion sou..."
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,Brown Sugar Meatloaf,"[packed brown sugar, ketchup, lean ground beef...","[ brown sugar, ketchup, lean ground beef, milk..."
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,Best Chocolate Chip Cookies,"[butter, softened, white sugar, packed brown s...","[butter , white sugar, brown sugar, eggs, van..."
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm,Homemade Mac and Cheese Casserole,"[whole wheat rotini pasta, fresh broccoli flor...","[whole wheat rotini pasta, fresh broccoli flor..."


In [4]:
def giveResults(query, last, wish,title,instructions,ingredients):
    res = es.search(index="recipes", body={"query": {"bool": { "must": [{"match":{ingredient_list:query} }]}},"size":max_res})
    mscores = [] # new scores for personalized results
    m2scores = [] #new scores for advanced personalized results
    results = [] #unpersonalized
    presults = [] #personalized 
    p2results = [] #adv personalized
    for hit in res['hits']['hits']:
        if title:
            results.append((hit['_source']['title'],hit['_score']))
        if ingredients:
            results.append((hit['_source'][ingredient_list]))
        if instructions: 
            results.append((hit['_source']['instructions']))
        ingr = ''
        for s in hit['_source'][ingredient_list]:
            ingr += s
        ldocuments = (ingr,last)
        wdocuments = (ingr,wish)
        tfidf_vectorizer=TfidfVectorizer()
        tfidf_last=tfidf_vectorizer.fit_transform(ldocuments)
        tfidf_wish=tfidf_vectorizer.fit_transform(wdocuments)
        cs = -100 * cosine_similarity(tfidf_last[0:1],tfidf_last)
        ds = 100 * cosine_similarity(tfidf_wish[0:1],tfidf_wish)
        mscores.append((hit['_source'],hit['_score']+cs[0][1]))
        m2scores.append((hit['_source'],hit['_score']+cs[0][1]+ds[0][1]))
    mscores.sort(key=lambda t: t[1],reverse=True)
    m2scores.sort(key=lambda t: t[1],reverse=True)
    for t in mscores:
        if title:
            presults.append((t[0]['title'],t[1]))
        if instructions:
            presults.append((t[0]['instructions'],t[1]))
        if ingredients:
            presults.append((t[0][ingredient_list],t[1]))
    for t in m2scores:
        if title:
            p2results.append((t[0]['title'],t[1]))
        if instructions:
            p2results.append((t[0]['instructions'],t[1]))
        if ingredients:
            p2results.append((t[0][ingredient_list],t[1]))
    return presults,p2results, results

In [5]:
def getInput():
    query = input("Enter whatever ingredients you have in your pantry seperated by whitespaces: ")
    last = input("Enter at least 5 ingredients you ate last week seperated by whitespace: ")
    wish = input("List anything you would like to eat. You can also just skip this.")
    return query,last,wish

In [6]:
def giveRecipes(title, instructions,ingredients):
    query,last,wish = getInput()
    print(query)
    personalized, personalized2, standard = giveResults(query,last,wish,title,instructions,ingredients)
    return personalized, personalized2, standard

In [18]:
p1,p2,s = giveRecipes(title=True,instructions=False,ingredients=True)

Enter whatever ingredients you have in your pantry seperated by whitespaces: paprika beans lentils tomatoes parmigiana cheese butter pasta rice meat kale spinach almonds 
Enter at least 5 ingredients you ate last week seperated by whitespace: asparagus couscous steak potatoes broccoli carote brussel+sprout  
List anything you would like to eat. You can also just skip this.soup
paprika beans lentils tomatoes parmigiana cheese butter pasta rice meat kale spinach almonds 


In [19]:
for res in s[:20]:
    print(res)#standard

('Greek Pasta with Tomatoes and White Beans', 18.30928)
['italian-style tomatoes', 'cannellini beans   ', 'fresh spinach  ', 'penne pasta', ' feta cheese']
('Harira', 16.801353)
[' lamb meat', 'ground turmeric', 'ground black pepper', 'ground cinnamon', 'ground ginger', 'ground  ', 'margarine', 'celery', 'onion', 'red onion', 'fresh cilantro', 'tomatoes', 'water', 'green lentils', 'garbanzo beans ', 'vermicelli pasta', 'eggs ', 'lemon ']
('10-Minute Ham, White Bean and Kale Salad', 15.969154)
['  white beans   ', ' deli meat such  ham  turkey', 'baby kale', ' cranberries', 'manchego cheese', 'dinner rolls  ', ' almonds', 'red wine vinegar', 'dijon mustard', 'kosher salt   ground black pepper', 'extra-virgin olive oil', '']
("Sarah's Feta Rice Pilaf", 15.507189)
['butter', 'orzo pasta', 'onion', 'cloves garlic ', 'white rice', 'chicken broth', 'spinach', 'bulgarian feta cheese']
('Tomato Orzo Soup with Kale', 15.2933655)
['beef stock', 'orzo pasta', 'fire-roasted tomatoes  juice', 'toma

In [21]:
for res in p1[:20]:
    print(res)#personalized using only last week

('Greek Pasta with Tomatoes and White Beans', 18.30928)
(['italian-style tomatoes', 'cannellini beans   ', 'fresh spinach  ', 'penne pasta', ' feta cheese'], 18.30928)
('Harira', 16.801353)
([' lamb meat', 'ground turmeric', 'ground black pepper', 'ground cinnamon', 'ground ginger', 'ground  ', 'margarine', 'celery', 'onion', 'red onion', 'fresh cilantro', 'tomatoes', 'water', 'green lentils', 'garbanzo beans ', 'vermicelli pasta', 'eggs ', 'lemon '], 16.801353)
('10-Minute Ham, White Bean and Kale Salad', 15.969154)
(['  white beans   ', ' deli meat such  ham  turkey', 'baby kale', ' cranberries', 'manchego cheese', 'dinner rolls  ', ' almonds', 'red wine vinegar', 'dijon mustard', 'kosher salt   ground black pepper', 'extra-virgin olive oil', ''], 15.969154)
("Sarah's Feta Rice Pilaf", 15.507189)
(['butter', 'orzo pasta', 'onion', 'cloves garlic ', 'white rice', 'chicken broth', 'spinach', 'bulgarian feta cheese'], 15.507189)
('Tomato Orzo Soup with Kale', 15.2933655)
(['beef stock',

In [22]:
for res in p2[:20]:
    print(res)#personalized using last week and wish

('Shrimp and Spinach Noodle Soup', 48.39046915910079)
(['water', 'pasta', 'packets  soup base', 'shrimp   ', 'spinach'], 48.39046915910079)
('Slow Cooker Manly Stew', 48.05559149521847)
(['condensed cream  mushroom soup', ' cheddar cheese soup', 'beef  onion soup mix', 'beef stew meat'], 48.05559149521847)
('Minestrone To Die For', 42.72052115091341)
([' stew meat  round steak', ' minestrone soup', ' tomato soup', 'water', 'dry onion soup mix'], 42.72052115091341)
('Slow Cooker Beef and Mushrooms', 41.82789697176422)
([' beef stew meat', ' golden mushroom soup', ' mushrooms ', 'dry onion soup mix', 'red wine', 'pinch black pepper  ', 'uncooked white rice', 'water'], 41.82789697176422)
('Jicama Spinach Dip', 41.30286397176422)
(['mayonnaise', 'containers  cream', 'dry leek soup mix', 'packet dry vegetable soup mix', 'paprika', ' ground black pepper', 'jicama   ', 'red bell pepper', 'green onions', 'frozen spinach   '], 41.30286397176422)
('Two by Four Stew', 40.678569832792604)
(['groun

In [5]:
import sys
print(sys.version)

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
